In [24]:
from scipy.io.wavfile import write, read
from scipy import signal
from scipy.fftpack import fft, ifft, fftshift, fftfreq
from scipy.io.wavfile import write, read
import matplotlib.pyplot as plt
import numpy as np
import sounddevice as sd

In [25]:
# Leitura do sinal da gaita

fs, gaita_in = read('../gaita.wav')

# Normalizando amplitude para valores entre -1 e 1 e definindo o tipo float 32bits
gaita_in = np.float32(gaita_in)/(2**15 - 1)

# Array de tempo
amostras = np.arange(len(gaita_in))
t = amostras / fs

# Obtendo a envoltória com filtro passa baixa
gaita_abs = abs(gaita_in)

fc = 30                #freq de corte
Wn = fc / (fs / 2)     #freq de corte normalizada

# Utilizando filtro butterworth de terceira ordem para aplicar um passa-baixa no sinal
b, a = signal.butter(3, Wn)
env_filt = signal.filtfilt(b, a, gaita_abs)

# Suprimir ruído desconsiderando amplitudes pequenas
for i in range (env_filt.size):
    if env_filt[i] < 0.005: env_filt[i] = 0

env_filt_ganho = 3.5 * env_filt

# Aplicando FFT no sinal da gaita
gaita_fft = fft(gaita_in)

# Definindo eixo das frequencias
# Está variando de 0 a fs/2 e pegando só metade do comprimento da fft, ou seja, só a porção positiva
gaita_freqs = np.linspace(0.0, fs/2, gaita_fft.size//2)

# Acertando a amplitude da fft
# Os valores absolutos da FFT dividido pela metade da frequencia de amostragem
amp_gaita_fft = abs(gaita_fft[:int(gaita_fft.size/2)]) / (fs/2)

# Obtendo o angulo da FFT
fase_gaita_fft = np.angle(gaita_fft[:int(gaita_fft.size/2)])

In [26]:
# Encontrando as frequências e amplitude relativa
pico_loc = []
gaita_fft_copy = amp_gaita_fft.copy()

# Laço para identificar a localização dos picos das ffts, desconsiderando os valores próximos
# Limiar para amplitude de 0.015
while True:
    pico = np.amax(gaita_fft_copy)
    if pico < 0.015: break
    loc, = np.where(gaita_fft_copy == pico)
    pico_loc.append(int(loc))
    gaita_fft_copy[int(loc-10):int(loc+10)] = 0

In [27]:
# Somando os senos mais significativos para formar os timbres das notas
# Para formar as outras notas da escala, utilizou-se a relação entre notas de acordo com a afinação natural

timbre_fase_C = 0
timbre_fase_D = 0
timbre_fase_E = 0
timbre_fase_F = 0
timbre_fase_G = 0
timbre_fase_A = 0
timbre_fase_B = 0

for loc in pico_loc:
    timbre_fase_C += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (8/9*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    timbre_fase_D += (amp_gaita_fft[loc] * np.cos(2 * np.pi * gaita_freqs[loc] * t + fase_gaita_fft[loc]))
    timbre_fase_E += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (10/9*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    timbre_fase_F += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (32/27*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    timbre_fase_G += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (4/3*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    timbre_fase_A += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (40/27*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    timbre_fase_B += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (5/3*gaita_freqs[loc]) * t + fase_gaita_fft[loc]))
    

In [35]:
sd.play(gaita_in)

In [30]:
# Multiplicando a envoltória ao timbre com um ganho

gaita_emuD = 3 * timbre_fase_D * env_filt_ganho

In [36]:
sd.play(gaita_emuD)

In [22]:
# estreitar a envoltória
fatorM = 4
env_short = np.array([])

for i in range(env_filt_ganho.size):
    if i%fatorM == 0:
        env_short = np.append(env_short, env_filt_ganho[i])
        
gaita_shortC = 3 * env_short * timbre_fase_C[:int(env_short.size)]

In [23]:
sd.play(gaita_shortC)

In [48]:
tshort = np.arange(env_short.size) / fs

timbre_short_C2 = 0
for loc in pico_loc:
    timbre_short_C2 += (amp_gaita_fft[loc] * np.cos(2 * np.pi * (8/9*gaita_freqs[loc]) * tshort + fase_gaita_fft[loc]))
    
gaita_shortC2 = 3 * env_short * timbre_short_C

In [51]:
sd.play(gaita_shortC2)

In [54]:
# alongar a envoltória
fatorL = 2
env_long = np.array([])

for i in range(env_filt_ganho.size):
    if i%fatorL == 0:
        env_long = np.append(env_long, env_filt_ganho[i])
    else:
        env_long = np.append(env_long, 0)

# plt.stem(env_long)
# plt.show()

In [70]:
teste = 3 * env_long * timbre_fase_A

In [72]:
sd.play(gaita_emuA)

In [73]:
sd.play(teste)